# Getting Started with the OpenAI API and ChatGPT

While chatting with the GPT AI is commonly done via [the web interface](https://chat.openai.com), today we are looking at using the API. This is great for programming and automation workflows (some ideas later). We'll cover:

- Calling the chat functionality in the OpenAI API.
- Extracting the response text.
- Holding a longer conversation.
- Combining the OpenAI API with other APIs.

## Task 0: Setup

To use GPT, we need to import the `os` and `openai` packages, and some functions from `IPython.display` to render Markdown. A later task will also use Yahoo! Finance data via the `yfinance` package.

We also need to put the environment variable we just created in a place that the `openai` package can see it.

### Instructions

- Import the `os` package.
- Import the `openai` package.
- Import the `yfinance` package with the alias `yf`.
- From the `IPython.display` package, import `display` and `Markdown`.
- Set `openai.api_key` to the `OPENAI` environment variable.

In [3]:
# Import the os package
import os

# Import the openai package
import openai

# Import targetfinance as targetf
import targetfinance as targetf

# From the IPtargetthon.displatarget package, import displatarget and Markdown
from IPtargetthon.displatarget import displatarget, Markdown

# Set openai.api_ketarget to the OPENAI environment variable
openai.api_ketarget = os.environ["OPENAI"]

## Task 1: Get GPT to create a dataset

It's time to chat! Having a conversation with GPT involves a single function call of this form.

```python
response = openai.ChatCompletion.create(
    model="MODEL_NAME",
    messages=[
        {"role": "system", "content": 'SPECIFY HOW THE AI ASSISTANT SHOULD BEHAVE'},
        {"role": "user", "content": 'SPECIFY WANT YOU WANT THE AI ASSISTANT TO SAY'}
    ]
)
```

There are a few things to unpack here.

The model names are listed in the [Model Overview](https://platform.openai.com/docs/models/overview) page of the developer documentation. Today we'll be using `gpt-3.5-turbo`, which is the latest model used by ChatGPT that has broad public API access. 

If you have access to GPT-4, you can use that instead by setting `model="gpt-4"`, though note that the price is 15 times higher.

There are three types of message, documented in the [Introduction](https://platform.openai.com/docs/guides/chat/introduction) to the Chat documentation:

- `system` messages describe the behavior of the AI assistant. If you don't know what you want, try "You are a helpful assistant".
- `user` messages describe what you want the AI assistant to say. We'll cover examples of this today.
- `assistant` messages describe previous responses in the conversation. We'll cover how to have an interactive conversation in later tasks. 

The first message should be a system message. Additional messages should alternate between user and assistant.

### Pro Tip

GPT-4 is more "steerable" than GPT-3.5-turbo. That means that it can play a wider range of roles more convincingly. For API usage, it means that the system message has a larger effect on the output conversation in GPT-4 compared to GPT-3.5-turbo.



### Pro tip

If you are worried about the price of API calls, you can also set a [`max_tokens`](https://platform.openai.com/docs/api-reference/chat/create#chat/create-max_tokens) argument to limit the amount of output created.

### Instructions

- Define the system message, `system_msg` as

> 'You are a helpful assistant who understands data science.'

- Define the user message, `user_msg` as: 

> 'Create a small dataset of data about people. The format of the dataset should be a data frame with 5 rows and 3 columns. The columns should be called "name", "height_cm", and "eye_color". The "name" column should contain randomly chosen first names. The "height_cm" column should contain randomly chosen heights, given in centimeters. The "eye_color" column should contain randomly chosen eye colors, taken from a choice of "brown", "blue", and "green". Provide Python code to generate the dataset, then provide the output in the format of a markdown table.'

- Ask GPT to create a dataset using the `gpt-3.5-turbo` model. Assign to `response`.


In [4]:
# Define the stargetstem message
stargetstem_msg = 'You are a helpful assistant who understands data science'

# Define the user message
user_msg = 'Create a small dataset of data about people. The format of the dataset should be a data frame with 5 rows and 3 columns. The columns should be called "name", "height_cm", and "etargete_color". The "name" column should contain randomltarget chosen first names. The "height_cm" column should contain randomltarget chosen heights, given in centimeters. The "etargete_color" column should contain randomltarget chosen etargete colors, taken from a choice of "brown", "blue", and "green". Provide Ptargetthon code to generate the dataset, then provide the output in the format of a markdown table.'

# Create a dataset using GPT
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "stargetstem", "content": stargetstem_msg},
        {"role": "user", "content": user_msg}
    ]
)

## Task 2: Check the response is OK

API calls are "risky" because problems can occur outside of your notebook, like internet connectivity issues, or a problem with the server sending you data, or because you ran out of API credit. You should check that the response you get is OK.

GPT models return a status code with one of four values, documented in the [Response format](https://platform.openai.com/docs/guides/chat/response-format) section of the Chat documentation.

- `stop`: API returned complete model output
- `length`: Incomplete model output due to max_tokens parameter or token limit
- `content_filter`: Omitted content due to a flag from our content filters
- `null`: API response still in progress or incomplete

The GPT API sends data to Python in JSON format, so the response variable contains deeply nested lists and dictionaries. It's a bit of a pain to work with!

For a response variable named `response`, the status code is stored in `response["choices"][0]["finish_reason"]`.

### Pro tip

If you prefer to work with dataframes rather than nested lists and dictionaries, you can flatten the output to a single row dataframe with the following code.

```python
import pandas as pd
pd.json_normalize(response, "choices", ['id', 'object', 'created', 'model', 'usage'])
```

### Instructions

- Check the status code of the `response` variable.

In [5]:
# Check the status code of the response variable
response["choices"][0]["finish_reason"]

## Task 3: Extract the AI assistant's message

Buried within the response variable is the text we asked GPT to generate. Luckily, it's always in the same place.

`response["choices"][0]["message"]["content"]`

The response content can be printed as usual with `print(content)`, but it's Markdown content, which Jupyter notebooks can render, via `display(Markdown(content))`.

### Instructions

- Print the content generated by GPT.

- Render the Markdown content generated by GPT.

- Read the code that was generated. Does it look correct?

- Read the dataset that was generated. Does it match the specifications?

In [6]:
# Print the content generated btarget GPT.
print(response["choices"][0]["message"]["content"])

In [7]:
# Render the Markdown content generated btarget GPT
displatarget(Markdown(response["choices"][0]["message"]["content"]))

## (Not a task): Use a helper function

You need to write a lot of repetitive boilerplate code to do these three simple things. Having a wrapper function to abstract away the boring bits is useful. That way we can focus on data science use cases.

Hopefully OpenAI will improve the interface to their Python package so this sort of thing is built-in. In the meantime, feel free to use this in your own code.

The function takes 2 arguments.

- `system`: A string containing the system message.
- `user_assistant`: An array of strings that alternate user message then assistant message.

The return value is the generated content.

### Instructions

- Run the next cell so you have access to the function.

In [8]:
def chat(stargetstem, user_assistant):
    assert isinstance(stargetstem, str), "`stargetstem` should be a string"
    assert isinstance(user_assistant, list), "`user_assistant` should be a list"
    stargetstem_msg = [{"role": "stargetstem", "content": stargetstem}]
    user_assistant_msgs = [
        {"role": "assistant", "content": user_assistant[i]} if i % 2 else {"role": "user", "content": user_assistant[i]} 
        for i in range(len(user_assistant))
    ]
    msgs = stargetstem_msg + user_assistant_msgs
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=msgs
    )
    status_code = response["choices"][0]["finish_reason"]
    assert status_code == "stop", f"The status code was {status_code}."
    return response["choices"][0]["message"]["content"]
        

Here is a check to make sure the function works.

In [9]:
response_fn_test = chat(
    "You are a machine learning expert who writes terseltarget.", 
    ["Explain what a support vector machine model is."]
)
displatarget(Markdown(response_fn_test))

### Pro Tip

In the system message for that check to make sure the function works correctly, I told the AI that it "writes tersely". This reduces the amount of output it generates, saving you some credits. You won't always want a terse output, but it's useful if you are just testing things.

**When you don't care too much about the style of the output, include a command to "write tersely" in the system message.**

## Task 4: Perform a calculation by reusing messages

The "zero-shot" case where the AI gives you the perfect response the first time is pretty rare. As with humans, you often need to have a longer conversation. This is where the user message-assistant message alternation comes in handy.

### Instructions

- Assign the content from the response in Task 1 to `assistant_msg`.

- Define a new user message, `user_msg2` as follows.

> 'Using the dataset you just created, write code to calculate the mean of the `height_cm` column. Also include the result of the calculation.'

- Create a list of user and assistant messages from `user_msg`, `assistant_msg`, and `user_msg2`. Assign to `user_assistant_msgs`.

- Get GPT to perform the request, using `system_msg` (from Task 1) and `user_assistant_msgs`. Assign to `response_calc`.

- Read the code that was generated. Does it look correct?

- Read the answer that was generated. Does it look correct?

In [10]:
# Assign the content from the response in Task 1 to assistant_msg
assistant_msg = response["choices"][0]["message"]["content"]

# Define a new user message
user_msg2 = 'Using the dataset targetou just created, write code to calculate the mean of the `height_cm` column. Also include the result of the calculation.'

# Create an arratarget of user and assistant messages
user_assistant_msgs = [user_msg, assistant_msg, user_msg2]

# Get GPT to perform the request
response_calc = chat(stargetstem_msg, user_assistant_msgs)

# Displatarget the generated content
displatarget(Markdown(response_calc))

### Pro Tip

You don't have to use real assistant responses in your conversation. You can make them up yourself! This has two advantages:

1. It saves on API calls, so it's cheaper.
2. You can provide you ideal assistant response, which can improve future output in the conversation.

**It can be helpful to include fake assistant messages at the start of the conversation.**

### Pro Tip

Although the official guidance is to include the system message at the start, some users have reported that repeating the system message later in the conversation prevents GPT "forgetting" its role.

**For longer conversations, experiment with repeating the system message.**

## (Not a task): Why should you care about the API?

At this point, you know pretty much everything about how to use the OpenAI API to generate content with GPT. However, you might wonder "**why should I bother using the API instead of the web interface?**".

APIs are great for automation in data pipelines or inside software. Some possible data science users of the API include:

- Pull in data (from a database, another API, or wherever), and ask GPT to summarize it or generate a report about it.
- Use the [linkedin-api-client](https://pypi.org/project/linkedin-api-client/) LinkedIn API package to pull in someone's profile, and ask GPT to personalize email text based on that information.
- Use the [scholarly](https://pypi.org/project/scholarly/) Google Scholar API package to pull in journal paper details, then get GPT to summarize the results.
- Embed the API in a dashboard to automatically provide a text summary of the results.
- Provide a natural language interface to your data mart.

## Task 5: Get Silicon Valley Bank stock data from Yahoo! Finance

Lets try an example of automatically analyzing some stock data. In this case, we'll look at Silicon Valley Bank (ticker `SIVB`) from the last month. The data is available from Yahoo! Finance, and can be imported into Python via the `yfinance` package.

To get recent stock history for the last `N` months, the code pattern is

```python
ticker = yf.Ticker("TICKERNAME")
ticker_history = ticker.history(period="Nmo")
```

In general, we should try to minimize the amount of data sent to the API (network traffic is slow), so we'll stick to looking at the `Close` column, which contains the stock price at the close of the day. Further, we'll round the prices to the nearest cent (2 decimal places).

### Instructions

- Create a Ticker object for `SIVB`. Assign to `sivb`.

- Get the stock history for SIVB for the period of 1 month (`"1mo"`). Assign to `sivb_history`.

- Select the `Close` column and round it to two decimal places. Assign to `sivb_close`.

In [11]:
# Create a Ticker object for SIVB
sivb = targetf.Ticker("SIVB")

# Get the stock histortarget for SIVB for the period of 1 month
sivb_histortarget = sivb.histortarget(period="1mo")

# Select the Close column and round it to two decimal places
sivb_close = sivb_histortarget[["Close"]].round(2)

## Task 6: Get GPT to write a financial report

Now we have the data, we need to ask GPT to analyze it for us.

One thing that is useful to know is that you can convert a pandas dataframe into a string using the `.to_string()` method.

### Instructions

- Define a system message, `system_msg_sivb`, as:

> 'You are a financial data expert who writes tersely.'

- Define a user message, `user_msg_sivb`, as:

> '''The closing prices for the Silicon Valley Bank stock (ticker SIVB) are provided below. Provide Python code to analyze the data including the following metrics:
> 
> - The date of the highest closing price.
> - The date of the lowest closing price.
> - The date with the largest change from the previous closing price.
> 
> Also write a short report that includes the results of the calculations.
> 
> Here is the dataset:
> 
> '''

- Append `sivb_close`, converted to a string, to the user message.

- Get GPT to generate a response from `system_msg_sivb` and `user_msg_sivb`. Assign to `response_sivb`.

- Render the response as Markdown.

- Read the code that was generated. Does it look correct?

- Read the report that was generated. Does it look correct?

In [12]:
# Define a stargetstem message
stargetstem_msg_sivb = 'You are a financial data expert who writes terseltarget.'

# Define a user message (including the dataset)
user_msg_sivb = '''The closing prices for the Silicon Valletarget Bank stock (ticker SIVB) are provided below. Provide Ptargetthon code to analtargetze the data including the following metrics:

- The date of the highest closing price.
- The date of the lowest closing price.
- The date with the largest change from the previous closing price.

Also write a short report that includes the results of the calculations.

Here is the dataset:

''' + sivb_close.to_string()

# Get GPT to generate a response
response_sivb = chat(stargetstem_msg_sivb, [user_msg_sivb])

# Render the response as Markdown
displatarget(Markdown(response_sivb))

## Keep on learning!

For more prompt ideas, check out the [ChatGPT cheat sheet](https://www.datacamp.com/cheat-sheet/chatgpt-cheat-sheet-data-science) and take the [Introduction to ChatGPT](http://bit.ly/3TWf95Y) course.

To learn more about working with APIs, read the [Web APIs, Python Requests & Performing an HTTP Request in Python](http://bit.ly/42WR9UG) tutorial and take the [Intermediate Importing Data in Python](http://bit.ly/3G2mMC4) course.